### Setup

In [1]:
import pandas as pd
match_path = '/Users/t_raver9/Desktop/projects/aflengine/analysis/machine_learning/src/player_data/data/matches_with_player_agg.csv'
players_path = '/Users/t_raver9/Desktop/projects/aflengine/analysis/machine_learning/src/player_data/data/players_with_player_stat_totals.csv'
matches = pd.read_csv(match_path)
players = pd.read_csv(players_path)

/Applications/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (15,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Applications/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (19,24,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Model Details

This model will be quite basic to serve as a baseline model. We'll be capturing:
 - form data for each team
 - career games played
 - only major indicators for individual data, such as supercoach points
 
The model will be a logistic regression.

In [32]:
features = [
    'diff_prem_points_form',
    'diff_percentage_form',
    'diff_season_Supercoach',
    'diff_season_clearances',
    'diff_career_goals',
    'diff_season_goals',
    'diff_season_hitouts',
    'diff_career_hitouts',
    'diff_season_inside50',
    'diff_season_rebound50',
    'diff_season_marks_in_50',
    'diff_season_tackles',
    'diff_season_disposal_efficiency',
    'diff_career_disposal_efficiency',
    'diff_career_games_played',
    'diff_season_games_played',
]

target = 'winner'

In [25]:
# Treat draws as away victories
model_data = matches[matches['season'] >= 2010]
for idx, row in model_data.iterrows():
    if row['winner'] == 'draw':
        model_data.at[idx,'winner'] = 'away'

Split into train and test data

In [30]:
rounds_in_season = get_season_rounds(model_data)
teams_in_season = get_season_teams(model_data)

In [31]:
# test set
from copy import deepcopy

test_data = pd.DataFrame()
for season, max_round in rounds_in_season.items():
    for rnd in range(1, max_round):
        round_matches = model_data[(model_data['season']==season) & (model_data['round']==rnd)]
        num_test = test_sample_size[len(round_matches)]
        round_test_set = round_matches.sample(num_test)
        test_data = test_data.append(round_test_set)
        
# training set
training_data = model_data.drop(test_data.index)

In [33]:
X_train = training_data[features]
y_train = training_data[target]
X_test = test_data[features]
y_test = test_data[target]

In [34]:
X_train.fillna(0,inplace=True)
y_train.fillna(0,inplace=True)
X_test.fillna(0,inplace=True)
y_test.fillna(0,inplace=True)

/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  **kwargs


### Modelling

In [22]:
import xgboost as xgb

Default parameters

In [39]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

xg_model_default = xgb.XGBClassifier()
kfold = KFold(n_splits=5, random_state=42)
results = cross_val_score(xg_model_default, X_train, y_train, cv=kfold)

print("Accuracy: %.2f%% (%.2f%% Std Dev)" % (results.mean()*100, results.std()*100))

Accuracy: 62.37% (4.00% Std Dev)


Tuning with grid search

In [43]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import numpy as np

xg_model_tuned = xgb.XGBClassifier()

param_grid = {
                 'nrounds': [50, 100, 200],
                 'eta': [.01, .1, .3],
                 'gamma': [0,1,10]
             }

grid_xgb = GridSearchCV(xg_model_tuned, param_grid, cv=5)
grid_xgb.fit(X_train, y_train)

[13:15:11] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { nrounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:15:11] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { nrounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:15:11] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { nrounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip throug

[13:15:15] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { nrounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:15:16] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { nrounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:15:16] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { nrounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip throug

[13:15:21] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { nrounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:15:22] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { nrounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:15:22] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { nrounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip throug

[13:15:26] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { nrounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:15:26] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { nrounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:15:26] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { nrounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip throug

[13:15:31] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { nrounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:15:31] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { nrounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:15:31] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { nrounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip throug

[13:15:37] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { nrounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:15:38] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { nrounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:15:38] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { nrounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip throug

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constrai...
                                     objective='binary:logistic',
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=None,
                                     subsample=None, tree_method=None,
                                   

In [45]:
grid_xgb.score(X_train,y_train)

0.719619326500732

In [46]:
grid_xgb.score(X_test,y_test)

0.6876061120543294

In [48]:
round1_2020 = model_data[model_data['season']==2020]
X = round1_2020[features]

In [53]:
grid_xgb.predict(X)

array(['home', 'home', 'away', 'away', 'home', 'away', 'home', 'home',
       'home'], dtype=object)

In [49]:
grid_xgb.predict_proba(X)

array([[0.3320878 , 0.6679122 ],
       [0.38158828, 0.6184117 ],
       [0.6908975 , 0.30910248],
       [0.5149505 , 0.4850495 ],
       [0.40655118, 0.5934488 ],
       [0.55451405, 0.44548598],
       [0.2831719 , 0.7168281 ],
       [0.3568558 , 0.6431442 ],
       [0.37384307, 0.6261569 ]], dtype=float32)

In [52]:
round1_2020[['hteam','ateam']]

,hteam,ateam
14950,Richmond,Carlton
14951,Footscray,Collingwood
14952,Gold Coast,Port Adelaide
14953,Adelaide,Sydney
14954,Essendon,Fremantle
14955,Greater Western Sydney,Geelong
14956,West Coast,Melbourne
14957,North Melbourne,St Kilda
14958,Hawthorn,Brisbane Lions


### Comparing with bookies

In [60]:
test_data[['homeodds','awayodds','winner']]

,homeodds,awayodds,winner
13008,2.64,1.49,away
13007,1.32,3.42,home
13015,1.63,2.30,away
13013,1.15,5.60,home
13020,1.46,2.73,away
...,...,...,...
14926,2.20,1.68,away
14928,1.22,4.30,home
14936,1.62,2.30,away
14933,1.95,1.85,home


Adding bookie info

In [76]:
test_data['book_pred'] = None
test_data['h_book_prob'] = None
test_data['a_book_prob'] = None
test_data['book_correct'] = None

for idx, row in test_data.iterrows():
    
    k = (1/row['homeodds'] + 1/row['awayodds'])
    test_data.at[idx, 'h_book_prob'] = k/row['homeodds']
    test_data.at[idx, 'a_book_prob'] = k/row['awayodds']
    
    if (row['homeodds'] >= row['awayodds']) & (row['winner'] == 'away'):
        test_data.at[idx,'book_pred'] = 'away'
        test_data.at[idx,'book_correct'] = True
    elif (row['homeodds'] > row['awayodds']) & (row['winner'] == 'home'):
        test_data.at[idx,'book_pred'] = 'away'
        test_data.at[idx,'book_correct'] = False
    if (row['homeodds'] <= row['awayodds']) & (row['winner'] == 'away'):
        test_data.at[idx,'book_pred'] = 'home'
        test_data.at[idx,'book_correct'] = False
    elif (row['homeodds'] < row['awayodds']) & (row['winner'] == 'home'):
        test_data.at[idx,'book_pred'] = 'home'
        test_data.at[idx,'book_correct'] = True

In [150]:
training_data['book_pred'] = None
training_data['h_book_prob'] = None
training_data['a_book_prob'] = None
training_data['book_correct'] = None

for idx, row in training_data.iterrows():
    
    k = (1/row['homeodds'] + 1/row['awayodds'])
    training_data.at[idx, 'h_book_prob'] = k/row['homeodds']
    training_data.at[idx, 'a_book_prob'] = k/row['awayodds']
    
    if (row['homeodds'] >= row['awayodds']) & (row['winner'] == 'away'):
        training_data.at[idx,'book_pred'] = 'away'
        training_data.at[idx,'book_correct'] = True
    elif (row['homeodds'] > row['awayodds']) & (row['winner'] == 'home'):
        training_data.at[idx,'book_pred'] = 'away'
        training_data.at[idx,'book_correct'] = False
    if (row['homeodds'] <= row['awayodds']) & (row['winner'] == 'away'):
        training_data.at[idx,'book_pred'] = 'home'
        training_data.at[idx,'book_correct'] = False
    elif (row['homeodds'] < row['awayodds']) & (row['winner'] == 'home'):
        training_data.at[idx,'book_pred'] = 'home'
        training_data.at[idx,'book_correct'] = True

In [83]:
sum(test_data['book_correct']==True)/len(test_data['book_correct'])

0.7368421052631579

In [151]:
sum(training_data['book_correct']==True)/len(training_data['book_correct'])

0.7005856515373353

In [97]:
test_data['book_correct'].value_counts()

True     434
False    153
Name: book_correct, dtype: int64

In [84]:
grid_xgb.score(X_test,y_test)

0.6876061120543294

Adding model odds info

In [105]:
model_odds = grid_xgb.predict_proba(X_test)
model_odds

array([[0.4349932 , 0.5650068 ],
       [0.44183147, 0.55816853],
       [0.48186564, 0.51813436],
       ...,
       [0.3598972 , 0.6401028 ],
       [0.35667044, 0.64332956],
       [0.5247855 , 0.4752145 ]], dtype=float32)

In [152]:
model_odds_train = grid_xgb.predict_proba(X_train)
model_odds_train

array([[0.45375794, 0.54624206],
       [0.37386572, 0.6261343 ],
       [0.4069693 , 0.5930307 ],
       ...,
       [0.2831719 , 0.7168281 ],
       [0.3568558 , 0.6431442 ],
       [0.37384307, 0.6261569 ]], dtype=float32)

In [109]:
a_model_odds = [x[0] for x in model_odds]
h_model_odds = [x[1] for x in model_odds]
model_pred = list(grid_xgb.predict(X_test))

In [155]:
a_model_odds_train = [x[0] for x in model_odds_train]
h_model_odds_train = [x[1] for x in model_odds_train]
model_pred_train = list(grid_xgb.predict(X_train))

In [127]:
test_data['h_model_prob'] = h_model_odds
test_data['a_model_prob'] = a_model_odds
test_data['model_pred'] = model_pred
test_data['model_correct'] = None
for idx, row in test_data.iterrows():
    if row['model_pred'] == row['winner']:
        test_data.at[idx,'model_correct'] = True
    else:
        test_data.at[idx,'model_correct'] = False

In [156]:
training_data['h_model_prob'] = h_model_odds_train
training_data['a_model_prob'] = a_model_odds_train
training_data['model_pred'] = model_pred_train
training_data['model_correct'] = None
for idx, row in training_data.iterrows():
    if row['model_pred'] == row['winner']:
        training_data.at[idx,'model_correct'] = True
    else:
        training_data.at[idx,'model_correct'] = False

In [128]:
test_data

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,...,book_pred,h_book_prob,a_book_prob,book_correct,a_model_odds,h_model_odds,model_pred,model_correct,h_model_prob,a_model_prob
13008,13008,13008,13008,13008,13008,13008,13008,13008,13008,9930,...,away,0.3977,0.70465,True,0.434993,0.565007,home,False,0.565007,0.434993
13007,13007,13007,13007,13007,13007,13007,13007,13007,13007,9929,...,home,0.795434,0.30701,True,0.441831,0.558169,home,True,0.558169,0.441831
13015,13015,13015,13015,13015,13015,13015,13015,13015,13015,11166,...,home,0.643116,0.455774,False,0.481866,0.518134,home,False,0.518134,0.481866
13013,13013,13013,13013,13013,13013,13013,13013,13013,13013,11164,...,home,0.911423,0.187167,True,0.347347,0.652653,home,True,0.652653,0.347347
13020,13020,13020,13020,13020,13020,13020,13020,13020,13020,556,...,home,0.720022,0.385067,False,0.372055,0.627945,home,False,0.627945,0.372055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14926,14926,14926,14926,14926,14926,14926,14926,14926,14926,671,...,away,0.477174,0.624871,True,0.507086,0.492914,away,True,0.492914,0.507086
14928,14928,14928,14928,14928,14928,14928,14928,14928,14928,1154,...,home,0.862484,0.244705,True,0.332088,0.667912,home,True,0.667912,0.332088
14936,14936,14936,14936,14936,14936,14936,14936,14936,14936,4114,...,home,0.649424,0.45742,False,0.359897,0.640103,home,False,0.640103,0.359897
14933,14933,14933,14933,14933,14933,14933,14933,14933,14933,4111,...,away,0.540185,0.569384,False,0.356670,0.643330,home,True,0.643330,0.356670


In [157]:
training_data

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,...,diff_career_games_played,diff_season_games_played,book_pred,h_book_prob,a_book_prob,book_correct,h_model_prob,a_model_prob,model_pred,model_correct
13004,13004,13004,13004,13004,13004,13004,13004,13004,13004,8872,...,11.817647,0.247059,away,0.363407,0.73961,True,0.546242,0.453758,home,False
13005,13005,13005,13005,13005,13005,13005,13005,13005,13005,9386,...,79.058824,4.882353,home,0.866668,0.233037,True,0.626134,0.373866,home,True
13006,13006,13006,13006,13006,13006,13006,13006,13006,13006,9928,...,7.122549,0.828431,away,0.212016,0.889388,True,0.593031,0.406969,home,False
13009,13009,13009,13009,13009,13009,13009,13009,13009,13009,10480,...,5.153846,0.138462,home,0.648262,0.454625,False,0.465132,0.534868,away,True
13010,13010,13010,13010,13010,13010,13010,13010,13010,13010,10481,...,7.456410,-1.758974,away,0.536094,0.56797,False,0.549659,0.450341,home,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14954,14954,14954,14954,14954,14954,14954,14954,14954,14954,6806,...,23.275735,2.033088,home,0.726408,0.383015,True,0.593449,0.406551,home,True
14955,14955,14955,14955,14955,14955,14955,14955,14955,14955,6807,...,-28.625000,-0.125000,home,0.700855,0.404339,True,0.445486,0.554514,away,False
14956,14956,14956,14956,14956,14956,14956,14956,14956,14956,7324,...,42.500000,2.066667,home,0.796737,0.309322,True,0.716828,0.283172,home,True
14957,14957,14957,14957,14957,14957,14957,14957,14957,14957,7325,...,11.126374,2.274725,away,0.527778,0.58642,False,0.643144,0.356856,home,True


Looks like, off the bat, we score worse than the bookies. What if we have a betting strategy of only betting on the favourite?

In [122]:
class Gambler:
    def __init__(self, funds):
        self.funds = funds
        self.current_bet = 0
        
    def bet(self, amount):
        if self.funds == 0:
            self.current_bet = 0
            print('Out of money!')
        elif amount >= self.funds:
            self.current_bet = self.funds
            self.funds = 0
        else:
            self.current_bet = amount
            self.funds -= amount
            
    def win(self, odds):
        self.funds += self.current_bet * odds

Strategy 1: start with $400, bet on the favourite each time

In [123]:
fav_total = []
fav_gambler = Gambler(400)
for idx, row in test_data.iterrows():
    fav_total.append(fav_gambler.funds)
    fav_gambler.bet(20)
    if (row['book_pred'] == row['winner']):
        if row['winner'] == 'home':
            fav_gambler.win(row['homeodds'])
        else:
            fav_gambler.win(row['awayodds'])

In [124]:
fav_total

[400,
 409.8,
 416.2,
 396.2,
 399.2,
 379.2,
 396.4,
 376.4,
 381.79999999999995,
 382.79999999999995,
 385.79999999999995,
 365.79999999999995,
 376.19999999999993,
 380.79999999999995,
 360.79999999999995,
 340.79999999999995,
 320.79999999999995,
 300.79999999999995,
 315.99999999999994,
 316.79999999999995,
 296.79999999999995,
 304.59999999999997,
 306.79999999999995,
 286.79999999999995,
 266.79999999999995,
 273.4,
 284.0,
 296.4,
 276.4,
 279.79999999999995,
 288.79999999999995,
 294.59999999999997,
 297.99999999999994,
 277.99999999999994,
 283.99999999999994,
 263.99999999999994,
 266.99999999999994,
 277.19999999999993,
 283.5999999999999,
 263.5999999999999,
 269.7999999999999,
 249.7999999999999,
 229.7999999999999,
 209.7999999999999,
 189.7999999999999,
 169.7999999999999,
 171.9999999999999,
 151.9999999999999,
 131.9999999999999,
 134.3999999999999,
 137.7999999999999,
 117.7999999999999,
 122.1999999999999,
 137.5999999999999,
 117.59999999999991,
 97.59999999999991,

Strategy 2: start with $400, bet on the model choice each time

In [125]:
model_total = []
model_gambler = Gambler(400)
for idx, row in test_data.iterrows():
    model_total.append(model_gambler.funds)
    model_gambler.bet(20)
    if (row['model_pred'] == row['winner']):
        if row['winner'] == 'home':
            model_gambler.win(row['homeodds'])
        else:
            model_gambler.win(row['awayodds'])

Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of money!
Out of

In [126]:
model_total

[400,
 380,
 386.4,
 366.4,
 369.4,
 349.4,
 366.59999999999997,
 346.59999999999997,
 351.99999999999994,
 352.99999999999994,
 355.99999999999994,
 388.99999999999994,
 368.99999999999994,
 373.59999999999997,
 353.59999999999997,
 333.59999999999997,
 313.59999999999997,
 293.59999999999997,
 308.79999999999995,
 309.59999999999997,
 289.59999999999997,
 297.4,
 299.59999999999997,
 279.59999999999997,
 304.2,
 310.8,
 321.40000000000003,
 333.80000000000007,
 313.80000000000007,
 317.20000000000005,
 326.20000000000005,
 332.00000000000006,
 335.40000000000003,
 315.40000000000003,
 321.40000000000003,
 301.40000000000003,
 304.40000000000003,
 314.6,
 321.0,
 347.6,
 353.8,
 333.8,
 313.8,
 293.8,
 318.6,
 298.6,
 300.8,
 280.8,
 260.8,
 263.2,
 266.59999999999997,
 246.59999999999997,
 250.99999999999997,
 230.99999999999997,
 210.99999999999997,
 190.99999999999997,
 192.2,
 195.2,
 209.2,
 189.2,
 203.39999999999998,
 206.79999999999998,
 186.79999999999998,
 190.99999999999997

Strategy 3: Only betting when the home team is expected to lose, but the our model gives the home team a better chance than the bookies. The bookies have underestimated the home team.

In [147]:
total_3 = []
gambler_3 = Gambler(400)
for idx, row in test_data.iterrows():
    if (row['book_pred'] == 'away'):
        if row['h_model_prob'] > row['h_book_prob']:
            if row['model_pred'] == 'home':
                total_3.append(gambler_3.funds)
                gambler_3.bet(20)
                if (row['model_pred'] == row['winner']):
                    if row['winner'] == 'home':
                        gambler_3.win(row['homeodds'])
                    else:
                        gambler_3.win(row['awayodds'])

In [148]:
total_3

[400,
 380,
 413.0,
 393.0,
 417.8,
 397.8,
 377.8,
 357.8,
 387.8,
 454.8,
 434.8,
 478.8,
 511.8,
 491.8,
 471.8,
 451.8,
 476.8,
 456.8,
 436.8,
 457.8,
 481.8,
 504.8,
 484.8,
 464.8,
 492.8,
 520.8,
 551.8,
 531.8,
 511.79999999999995,
 491.79999999999995,
 471.79999999999995,
 451.79999999999995,
 485.79999999999995,
 465.79999999999995,
 489.79999999999995,
 469.79999999999995,
 449.79999999999995,
 429.79999999999995,
 458.79999999999995,
 438.79999999999995,
 418.79999999999995,
 398.79999999999995,
 378.79999999999995,
 398.79999999999995,
 378.79999999999995,
 358.79999999999995,
 338.79999999999995,
 360.79999999999995,
 388.79999999999995,
 368.79999999999995,
 413.79999999999995,
 393.79999999999995,
 373.79999999999995,
 353.79999999999995,
 333.79999999999995,
 352.99999999999994,
 332.99999999999994,
 353.3999999999999,
 376.3999999999999,
 356.3999999999999,
 336.3999999999999,
 379.3999999999999,
 433.3999999999999,
 477.3999999999999,
 507.3999999999999,
 487.399999

Do this with training data to get a larger sample

In [165]:
total_3_train = []
gambler_3_train = Gambler(400)
for idx, row in training_data.iterrows():
    if (row['book_pred'] == 'away'):
        if row['h_model_prob'] > row['h_book_prob']:
            if row['model_pred'] == 'home':
                total_3_train.append(gambler_3_train.funds)
                gambler_3_train.bet(20)
                if (row['model_pred'] == row['winner']):
                    if row['winner'] == 'home':
                        gambler_3_train.win(row['homeodds'])
                    else:
                        gambler_3_train.win(row['awayodds'])

In [166]:
total_3_train

[400,
 380,
 360,
 379.2,
 405.4,
 428.2,
 466.0,
 492.6,
 514.6,
 494.6,
 516.4,
 496.4,
 528.6,
 508.6,
 488.6,
 527.6,
 560.6,
 579.2,
 559.2,
 539.2,
 519.2,
 499.20000000000005,
 521.8000000000001,
 501.80000000000007,
 481.80000000000007,
 542.8000000000001,
 575.8000000000001,
 595.8000000000001,
 641.8000000000001,
 621.8000000000001,
 601.8000000000001,
 624.8000000000001,
 604.8000000000001,
 584.8000000000001,
 605.8000000000001,
 585.8000000000001,
 565.8000000000001,
 545.8000000000001,
 525.8000000000001,
 593.8000000000001,
 573.8000000000001,
 553.8000000000001,
 589.8000000000001,
 609.8000000000001,
 589.8000000000001,
 569.8000000000001,
 549.8000000000001,
 609.8000000000001,
 637.8000000000001,
 671.8000000000001,
 651.8000000000001,
 682.8000000000001,
 702.8000000000001,
 682.8000000000001,
 662.8000000000001,
 691.8000000000001,
 713.4000000000001,
 737.4000000000001,
 717.4000000000001,
 777.4000000000001,
 757.4000000000001,
 789.4000000000001,
 769.4000000000

Need to check this against the other models you ran, i.e. always choose the model

In [170]:
model_total_train = []
model_gambler_train = Gambler(400)
for idx, row in training_data.iterrows():
    model_total_train.append(model_gambler_train.funds)
    model_gambler_train.bet(20)
    if (row['model_pred'] == row['winner']):
        if row['winner'] == 'home':
            model_gambler_train.win(row['homeodds'])
        else:
            model_gambler_train.win(row['awayodds'])

In [172]:
model_total_train[-1]

2378.6000000000017

Suddenly it doesn't look so good. But useful to compare at a per-game basis

In [173]:
len(total_3_train)

137

In [178]:
model_total_train[136]

723.1999999999995

In [177]:
total_3_train[136]

1390.6000000000001

Seems like it might be worth a shot?

### Metadata and Functions

In [2]:
player_cols = ['AFLfantasy',
 'Supercoach',
 'behinds',
 'bounces',
 'brownlow',
 'clangers',
 'clearances',
 'contested_marks',
 'contested_poss',
 'disposals',
 'frees_against',
 'frees_for',
 'goal_assists',
 'goals',
 'handballs',
 'hitouts',
 'inside50',
 'kicks',
 'marks',
 'marks_in_50',
 'one_percenters',
 'rebound50',
 'tackles',
 'tog',
 'uncontested_poss',
 'centre_clearances',
 'disposal_efficiency',
 'effective_disposals',
 'intercepts',
 'metres_gained',
 'stoppage_clearances',
 'score_involvements',
 'tackles_in_50',
 'turnovers',
 'matchid',
 'next_matchid',
 'team',
 'career_AFLfantasy',
 'career_Supercoach',
 'career_behinds',
 'career_bounces',
 'career_brownlow',
 'career_clangers',
 'career_clearances',
 'career_contested_marks',
 'career_contested_poss',
 'career_disposals',
 'career_frees_against',
 'career_frees_for',
 'career_goal_assists',
 'career_goals',
 'career_handballs',
 'career_hitouts',
 'career_inside50',
 'career_kicks',
 'career_marks',
 'career_marks_in_50',
 'career_one_percenters',
 'career_rebound50',
 'career_tackles',
 'career_tog',
 'career_uncontested_poss',
 'career_centre_clearances',
 'career_disposal_efficiency',
 'career_effective_disposals',
 'career_intercepts',
 'career_metres_gained',
 'career_stoppage_clearances',
 'career_score_involvements',
 'career_tackles_in_50',
 'career_turnovers',
 'season_AFLfantasy',
 'season_Supercoach',
 'season_behinds',
 'season_bounces',
 'season_brownlow',
 'season_clangers',
 'season_clearances',
 'season_contested_marks',
 'season_contested_poss',
 'season_disposals',
 'season_frees_against',
 'season_frees_for',
 'season_goal_assists',
 'season_goals',
 'season_handballs',
 'season_hitouts',
 'season_inside50',
 'season_kicks',
 'season_marks',
 'season_marks_in_50',
 'season_one_percenters',
 'season_rebound50',
 'season_tackles',
 'season_tog',
 'season_uncontested_poss',
 'season_centre_clearances',
 'season_disposal_efficiency',
 'season_effective_disposals',
 'season_intercepts',
 'season_metres_gained',
 'season_stoppage_clearances',
 'season_score_involvements',
 'season_tackles_in_50',
 'season_turnovers',
 'career_games_played',
 'season_games_played']

In [29]:
from typing import Dict
import numpy as np

def get_season_rounds(matches: pd.DataFrame) -> Dict:
    """
    Return a dictionary with seasons as keys and number of games
    in season as values
    """
    seasons = matches['season'].unique()
    rounds_in_season = dict.fromkeys(seasons,0)
    
    for season in seasons:
        rounds_in_season[season] = max(matches[matches['season']==season]['round'])
    
    return rounds_in_season

# What teams participated in each season?
def get_season_teams(matches: pd.DataFrame) -> Dict:
    """
    Return a dictionary with seasons as keys and a list of teams who played
    in that season as values
    """
    seasons = matches['season'].unique()
    teams_in_season = {}

    for season in seasons:
        teams = list(matches[matches['season']==season]['hteam'].unique())
        teams.extend(list(matches[matches['season']==season]['ateam'].unique()))
        teams = np.unique(teams)
        teams_in_season[season] = list(teams)
        
    return teams_in_season